In [8]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
import sys
sys.path.append('.')
import utils_original

In [9]:
X_train = pd.read_csv('./time_split/train_data_6_10_feature_selected_median_level_6.csv')
X_val = pd.read_csv('./time_split/val_data_11_feature_selected_median_level_6.csv')
X_test = pd.read_csv('./time_split/test_data_feature_selected_12_median_level_6.csv')
y_train = X_train.pop('label').as_matrix()
y_val = X_val.pop('label').as_matrix()
y_test = X_test.pop('label').as_matrix()
X_train = X_train.as_matrix()
X_val = X_val.as_matrix()
X_test = X_test.as_matrix()

min_max_scaler = MinMaxScaler()
min_max_scaler.fit(X_train)
X_train = min_max_scaler.transform(X_train)
X_val = min_max_scaler.transform(X_val)
X_test = min_max_scaler.transform(X_test)

In [11]:
train_val_features = np.concatenate((X_train, X_val), axis = 0)
train_val_labels = np.concatenate((y_train, y_val), axis = 0)

In [12]:
dtrain = xgb.DMatrix(train_val_features, label=train_val_labels) # xgboost data style
dval = xgb.DMatrix(X_val, label=y_val)
dtest = xgb.DMatrix(X_test, label=y_test)
params = {
    'booster': 'gbtree',
    'objective': 'binary:logitraw',
    'eval_metric': 'auc',
    'max_depth': 3,
    'max_delta_step': 3,
    'lambda': 1,
    'subsample': 0.5,
    'colsample_bytree': 0.8,
    'min_child_weight': 2,
    'eta': 0.1,
    'silent': 0,
    'scale_pos_weight': 3,
    'gamma': 0.1
}
watchlist = [(dtrain, 'train'), (dval, 'val'), (dtest, 'test')]
xg_model = xgb.train(params, dtrain, num_boost_round=100)

y_train_pred_prob = xg_model.predict(dtrain)
y_val_pred_prob = xg_model.predict(dval)
y_test_pred_prob = xg_model.predict(dtest)
utils_original.model_key_performance(y_train_pred_prob, train_val_labels)
utils_original.model_key_performance(y_val_pred_prob, y_val)
utils_original.model_key_performance(y_test_pred_prob, y_test)


percision: 0.5833 recall: 0.0136 f1: 0.0266 auc: 0.8855 ks: 0.6242 confusion_matrix: 
56420  5      
507    7      

percision: 1.0000 recall: 0.0247 f1: 0.0482 auc: 0.8797 ks: 0.6047 confusion_matrix: 
13943  0      
79     2      


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)



percision: 0.0000 recall: 0.0000 f1: 0.0000 auc: 0.7831 ks: 0.4818 confusion_matrix: 
21966  0      
149    0      
